# Group Project
#### **FINN43815 - Python for Finance**

In [59]:
# Importing necessary packages
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import kurtosis, skew
import yfinance as yf

### Momentum and HML

In [60]:
data = pd.read_csv('cleaned_data.csv')
data = data[["date", "permno", "price", "p2b"]]

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_3999/2384377179.py:1: DtypeWarning:

Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.



#### 2. Calculate returns:
- Formation period returns ('form_rets'): returns over the past J=6 months.
- Trading period returns ('hold_rets'): returns over the next K=1 month.

In [61]:
data['form_rets'] = data.groupby('permno')['price'].transform(pd.Series.pct_change, periods=6, fill_method=None)
data.head(20)

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_3999/1708375802.py:1: FutureWarning:

The 'fill_method' and 'limit' keywords in Series.pct_change are deprecated and will be removed in a future version. Call ffill before calling pct_change instead.



,date,permno,price,p2b,form_rets
0,1990-01-31,0111145D UN Equity,20.6875,NaN,NaN
1,1990-02-28,0111145D UN Equity,22.0000,NaN,NaN
2,1990-03-30,0111145D UN Equity,19.5000,NaN,NaN
3,1990-04-30,0111145D UN Equity,17.6250,1.5612,NaN
4,1990-05-31,0111145D UN Equity,19.8125,1.7550,NaN
5,1990-06-29,0111145D UN Equity,18.9375,1.6775,NaN
6,1990-07-31,0111145D UN Equity,19.6250,1.7262,-0.051360
7,1990-08-31,0111145D UN Equity,20.3750,1.7922,-0.073864
8,1990-09-28,0111145D UN Equity,21.7500,1.9131,0.115385
9,1990-10-31,0111145D UN Equity,22.8750,2.0292,0.297872


In [62]:
data['hold_rets'] = data.groupby('permno')['price'].pct_change(periods=1).groupby(data['permno']).shift(-2)
data.head(20)

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_3999/2474077109.py:1: FutureWarning:

The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.



,date,permno,price,p2b,form_rets,hold_rets
0,1990-01-31,0111145D UN Equity,20.6875,NaN,NaN,-0.113636
1,1990-02-28,0111145D UN Equity,22.0000,NaN,NaN,-0.096154
2,1990-03-30,0111145D UN Equity,19.5000,NaN,NaN,0.124113
3,1990-04-30,0111145D UN Equity,17.6250,1.5612,NaN,-0.044164
4,1990-05-31,0111145D UN Equity,19.8125,1.7550,NaN,0.036304
5,1990-06-29,0111145D UN Equity,18.9375,1.6775,NaN,0.038217
6,1990-07-31,0111145D UN Equity,19.6250,1.7262,-0.051360,0.067485
7,1990-08-31,0111145D UN Equity,20.3750,1.7922,-0.073864,0.051724
8,1990-09-28,0111145D UN Equity,21.7500,1.9131,0.115385,0.010929
9,1990-10-31,0111145D UN Equity,22.8750,2.0292,0.297872,-0.048649


#### 3. Define the stock universe and sort stocks into 3 decile groups

In [63]:
# Construct an indicator for a 'good' stock
data['good_stock'] = 0
data.loc[(~data['form_rets'].isnull()) & (~data['p2b'].isnull()), 'good_stock']=1
data.head(20)

,date,permno,price,p2b,form_rets,hold_rets,good_stock
0,1990-01-31,0111145D UN Equity,20.6875,NaN,NaN,-0.113636,0
1,1990-02-28,0111145D UN Equity,22.0000,NaN,NaN,-0.096154,0
2,1990-03-30,0111145D UN Equity,19.5000,NaN,NaN,0.124113,0
3,1990-04-30,0111145D UN Equity,17.6250,1.5612,NaN,-0.044164,0
4,1990-05-31,0111145D UN Equity,19.8125,1.7550,NaN,0.036304,0
5,1990-06-29,0111145D UN Equity,18.9375,1.6775,NaN,0.038217,0
6,1990-07-31,0111145D UN Equity,19.6250,1.7262,-0.051360,0.067485,1
7,1990-08-31,0111145D UN Equity,20.3750,1.7922,-0.073864,0.051724,1
8,1990-09-28,0111145D UN Equity,21.7500,1.9131,0.115385,0.010929,1
9,1990-10-31,0111145D UN Equity,22.8750,2.0292,0.297872,-0.048649,1


In [64]:
# Assign stocks into 3 equal groups based on formation period returns
data['port_mom'] = data.loc[data['good_stock']==1].groupby('date')['form_rets'].transform(pd.qcut, q=3, labels=range(1,4)).astype(str)
data

,date,permno,price,p2b,form_rets,hold_rets,good_stock,port_mom
0,1990-01-31,0111145D UN Equity,20.6875,NaN,NaN,-0.113636,0,NaN
1,1990-02-28,0111145D UN Equity,22.0000,NaN,NaN,-0.096154,0,NaN
2,1990-03-30,0111145D UN Equity,19.5000,NaN,NaN,0.124113,0,NaN
3,1990-04-30,0111145D UN Equity,17.6250,1.5612,NaN,-0.044164,0,NaN
4,1990-05-31,0111145D UN Equity,19.8125,1.7550,NaN,0.036304,0,NaN
...,...,...,...,...,...,...,...,...
514691,2022-04-29,J UN Equity,138.5500,2.9386,-0.013317,-0.092512,1,2
514692,2022-05-31,J UN Equity,140.0900,2.9712,-0.017326,0.079997,1,2
514693,2022-06-30,J UN Equity,127.1300,2.6964,-0.086907,-0.093664,1,3
514694,2022-07-29,J UN Equity,137.3000,2.9522,0.054694,NaN,1,3


In [65]:
# Assign stocks into 3 equal groups based on b/m ratio
data['port_value'] = data.loc[data['good_stock']==1].groupby('date')['p2b'].transform(pd.qcut, q=3, labels=range(1,4)).astype(str)
data

,date,permno,price,p2b,form_rets,hold_rets,good_stock,port_mom,port_value
0,1990-01-31,0111145D UN Equity,20.6875,NaN,NaN,-0.113636,0,NaN,NaN
1,1990-02-28,0111145D UN Equity,22.0000,NaN,NaN,-0.096154,0,NaN,NaN
2,1990-03-30,0111145D UN Equity,19.5000,NaN,NaN,0.124113,0,NaN,NaN
3,1990-04-30,0111145D UN Equity,17.6250,1.5612,NaN,-0.044164,0,NaN,NaN
4,1990-05-31,0111145D UN Equity,19.8125,1.7550,NaN,0.036304,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
514691,2022-04-29,J UN Equity,138.5500,2.9386,-0.013317,-0.092512,1,2,2
514692,2022-05-31,J UN Equity,140.0900,2.9712,-0.017326,0.079997,1,2,2
514693,2022-06-30,J UN Equity,127.1300,2.6964,-0.086907,-0.093664,1,3,2
514694,2022-07-29,J UN Equity,137.3000,2.9522,0.054694,NaN,1,3,2


#### 4. Find the intersection portfolios

In [66]:
data['port_comb'] = ''
data.loc[(data['port_mom']=='1') & (data['port_value']=='3'),'port_comb']='short_leg'
data.loc[(data['port_mom']=='3') & (data['port_value']=='1'),'port_comb']='long_leg'
data = data.loc[data['port_comb'].isin(['short_leg','long_leg'])]
data.head(20)

,date,permno,price,p2b,form_rets,hold_rets,good_stock,port_mom,port_value,port_comb
436,1993-09-30,0202445Q UN Equity,14.5901,1.7869,0.239432,-0.007199,1,3,1,long_leg
450,1994-11-30,0202445Q UN Equity,21.1668,3.3611,-0.092414,0.018090,1,1,3,short_leg
451,1994-12-30,0202445Q UN Equity,21.3878,3.3962,-0.082939,0.078678,1,1,3,short_leg
452,1995-01-31,0202445Q UN Equity,21.7747,4.5528,-0.061904,0.049413,1,1,3,short_leg
453,1995-02-28,0202445Q UN Equity,23.4879,4.9110,-0.049219,0.100899,1,1,3,short_leg
466,1996-03-29,0202445Q UN Equity,33.8226,6.6101,0.020000,0.047761,1,1,3,short_leg
470,1996-07-31,0202445Q UN Equity,30.1751,5.7978,-0.201753,0.032679,1,1,3,short_leg
471,1996-08-30,0202445Q UN Equity,33.8226,6.4986,-0.138027,-0.113923,1,1,3,short_leg
473,1996-10-31,0202445Q UN Equity,30.9488,5.4640,-0.164178,0.012232,1,1,3,short_leg
474,1996-11-29,0202445Q UN Equity,36.1438,6.3812,-0.068375,0.033231,1,1,3,short_leg


#### 5. Calculate trading period returns for long and short legs separately

In [67]:
port_comb_df = data.loc[~data['port_comb'].isnull()].groupby(['date','port_comb'])['hold_rets'].mean().reset_index()
port_comb_df.head(20)

,date,port_comb,hold_rets
0,1990-07-31,long_leg,-0.092128
1,1990-07-31,short_leg,-0.085286
2,1990-08-31,long_leg,0.013928
3,1990-08-31,short_leg,-0.037838
4,1990-09-28,long_leg,0.026538
5,1990-09-28,short_leg,0.171646
6,1990-10-31,long_leg,0.024719
7,1990-10-31,short_leg,0.041056
8,1990-11-30,long_leg,0.032136
9,1990-11-30,short_leg,0.164149


#### 6. Calculate the returns on long/short strategy

In [68]:
# Pivot the table
port_comb_df = port_comb_df.pivot(index = 'date', columns = 'port_comb', values = 'hold_rets').reset_index()
# Add column with long/short strategy returns
port_comb_df['Strat_rets'] = port_comb_df['long_leg']-port_comb_df['short_leg']
port_comb_df.head(20)

port_comb,date,long_leg,short_leg,Strat_rets
0,1990-07-31,-0.092128,-0.085286,-0.006842
1,1990-08-31,0.013928,-0.037838,0.051766
2,1990-09-28,0.026538,0.171646,-0.145109
3,1990-10-31,0.024719,0.041056,-0.016336
4,1990-11-30,0.032136,0.164149,-0.132013
5,1990-12-31,0.047656,0.115636,-0.067980
6,1991-01-31,0.088850,0.015670,0.073180
7,1991-02-28,0.021811,0.012987,0.008824
8,1991-03-28,0.071551,0.042626,0.028925
9,1991-04-30,-0.060025,-0.050289,-0.009736


#### 7. Plot the cumulative returns on the strategy

In [69]:
port_comb_df['Strat_cum_rets'] = (1+port_comb_df['Strat_rets']).cumprod()
px.line(port_comb_df, y='Strat_cum_rets', x='date', labels={'Strat_cum_rets': 'Cumulative returns'}, title = 'Value-Momentum Strategy').show()

In [70]:
px.histogram(port_comb_df[['Strat_rets']]).show()

In [73]:
port_comb_df.date = pd.to_datetime(port_comb_df.date)

In [74]:
monthly_rets = port_comb_df[["date", "Strat_rets"]].resample('M', on='date').mean() # Calculate average daily return for each month
monthly_rets['Colour']='red' # Add column with 'red' value
monthly_rets.loc[monthly_rets['Strat_rets']>=0, 'Colour']='green' # Replace 'red' with 'green' where return is non-negative
px.bar(monthly_rets,x=monthly_rets.index,y="Strat_rets").update_traces(marker_color=monthly_rets["Colour"]).show()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [75]:
# mean and volatility
mean_ret = port_comb_df['Strat_rets'].mean()
volatility = port_comb_df['Strat_rets'].std()
print(mean_ret, volatility)

-0.0017028237948208617 0.05126517933046392


In [76]:
print('Annualised expected return is: '+str(round(100*mean_ret*12,2)) +'%')
print('Annualised volatility is: '+str(round(100*volatility*np.sqrt(12),2)) +'%')

Annualised expected return is: -2.04%
Annualised volatility is: 17.76%


In [77]:
# describing statistics of returns
(port_comb_df['Strat_rets']*100).describe()

count    384.000000
mean      -0.170282
std        5.126518
min      -26.596846
25%       -2.339218
50%       -0.148853
75%        2.497987
max       19.807262
Name: Strat_rets, dtype: float64

In [78]:
# skewness
print(skew(port_comb_df['Strat_rets'].dropna()))

-0.5185975524269563


In [79]:
# kurtosis
print(kurtosis(port_comb_df["Strat_rets"].dropna()))

3.566801942456803


In [80]:
# Value at Risk
VaR = port_comb_df['Strat_rets'].quantile(0.005)
VaR

-0.17762862359418297

In [81]:
for q in [0.1, 0.05, 0.01, 0.001, 0.0001]:
    VaR = port_comb_df['Strat_rets'].quantile(q)
    print('There is a '+str(round(100*q,3))+'% chance that my losses will be '+str(round(VaR*100, 2))+'% or worse over the next day.')

There is a 10.0% chance that my losses will be -5.33% or worse over the next day.
There is a 5.0% chance that my losses will be -7.51% or worse over the next day.
There is a 1.0% chance that my losses will be -15.15% or worse over the next day.
There is a 0.1% chance that my losses will be -23.44% or worse over the next day.
There is a 0.01% chance that my losses will be -26.28% or worse over the next day.


In [82]:
VaR = port_comb_df['Strat_rets'].quantile(0.005)
rets_tail = port_comb_df.loc[port_comb_df['Strat_rets']<=VaR,["date", "Strat_rets", "Strat_cum_rets"]]
rets_tail

port_comb,date,Strat_rets,Strat_cum_rets
124,2000-11-30,-0.183477,0.518329
223,2009-02-27,-0.265968,0.557476


In [83]:
ES = rets_tail['Strat_rets'].mean()
ES

-0.22472282565718035

In [84]:
for q in [0.1, 0.05, 0.01, 0.001, 0.0001]:
    VaR = port_comb_df['Strat_rets'].quantile(q)
    rets_tail = port_comb_df.loc[port_comb_df['Strat_rets']<=VaR,:]
    ES = rets_tail['Strat_rets'].mean()
    print('I expect to lose on average '+ str(round(ES*100, 2))+'% over 1-day period given that I have exceeded my VaR for '+str(round(100*q,3))+'%.')

I expect to lose on average -9.95% over 1-day period given that I have exceeded my VaR for 10.0%.
I expect to lose on average -13.39% over 1-day period given that I have exceeded my VaR for 5.0%.
I expect to lose on average -19.62% over 1-day period given that I have exceeded my VaR for 1.0%.
I expect to lose on average -26.6% over 1-day period given that I have exceeded my VaR for 0.1%.
I expect to lose on average -26.6% over 1-day period given that I have exceeded my VaR for 0.01%.


In [85]:
# according to bloomberg the risk free rate (treasury yields) 4.76 % (last 30 years)
# calculation of sharpe ratio

rf = 0.0476
sharpe_ratio = (mean_ret-rf)/volatility
sharpe_ratio

-0.9617214733026996

In [86]:
# get SP500 data from 1990-01-31 to 2022-08-30	
sp500_ticker = yf.Ticker("^GSPC")
sp500 = sp500_ticker.history(start="1990-02-28", end="2022-09-30", interval="1mo")
sp500 = sp500[["Close"]]
sp500["Rets"] = sp500.Close.pct_change()

In [87]:
sp500

,Close,Rets
Date,,
1990-03-01 00:00:00-05:00,339.940002,NaN
1990-04-01 00:00:00-05:00,330.799988,-0.026887
1990-05-01 00:00:00-04:00,361.230011,0.091989
1990-06-01 00:00:00-04:00,358.019989,-0.008886
1990-07-01 00:00:00-04:00,356.149994,-0.005223
...,...,...
2022-05-01 00:00:00-04:00,4132.149902,0.000053
2022-06-01 00:00:00-04:00,3785.379883,-0.083920
2022-07-01 00:00:00-04:00,4130.290039,0.091116


In [88]:
# Calculate the average return of the market
average_market_return = sp500.Rets.mean()

# Calculate the covariance between portfolio and market returns
covariance = port_comb_df['Strat_rets'].cov(sp500['Rets'].reset_index(drop=True))

# Calculate the variance of the market returns
market_variance = sp500['Rets'].var()

# Calculate the portfolio's beta
portfolio_beta = covariance / market_variance

# Calculate Jensen's Alpha
jensens_alpha = mean_ret - (rf + portfolio_beta * (average_market_return - rf))

print("Portfolio Beta:", portfolio_beta)
print("Jensen's Alpha:", jensens_alpha)

Portfolio Beta: 0.15555927248467138
Jensen's Alpha: -0.04298512064864264


In [89]:
# treynor ratio
treynor_ratio = (mean_ret - rf)/portfolio_beta
treynor_ratio

-0.3169391512786813

In [90]:
# maximum drawdown
peak = port_comb_df.reset_index(drop=True).loc[1, "Strat_cum_rets"]
max_drawdown = 0
for ret in port_comb_df["Strat_cum_rets"].dropna():
    if ret >= peak:
        peak = ret
    else:
        drawdown = (peak - ret)/peak
        max_drawdown = max(max_drawdown, drawdown)

print(max_drawdown)

0.8449390312016193


In [91]:
# calmar-ratio
annualised_exp_return = round(mean_ret*12,2)
calmar_ratio = (annualised_exp_return - rf)/max_drawdown
calmar_ratio

-0.08000577261043738